In [21]:
import os
os.environ['KAGGLE_USERNAME'] = 'your_kaggle_username'
os.environ['KAGGLE_KEY'] = 'KGAT_5f85f9512fddf54ac2a8b6cfd5a03a87'


In [22]:
!pip install -q kaggle


In [23]:
!kaggle datasets list


ref                                                           title                                                     size  lastUpdated                 downloadCount  voteCount  usabilityRating  
------------------------------------------------------------  --------------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
saidaminsaidaxmadov/chocolate-sales                           Chocolate Sales                                         468320  2026-01-04 14:23:35.490000           9011        146  1.0              
vishardmehta/delhi-pollution-aqi-dataset                      Delhi Pollution AQI dataset                            5299345  2026-01-26 14:14:04.553000            879         27  1.0              
ayeshaimran1619/customer-spending-patterns                    Customer Spending Patterns                               72157  2026-01-24 14:56:48.767000            733         23  1.0              
aliiihussa

In [24]:
!kaggle datasets download -d wordsforthewise/lending-club
!unzip lending-club.zip -d data


Dataset URL: https://www.kaggle.com/datasets/wordsforthewise/lending-club
License(s): CC0-1.0
 99% 1.25G/1.26G [00:16<00:00, 102MB/s] 
100% 1.26G/1.26G [00:19<00:00, 71.3MB/s]
Archive:  lending-club.zip
  inflating: data/accepted_2007_to_2018Q4.csv.gz  
  inflating: data/accepted_2007_to_2018q4.csv/accepted_2007_to_2018Q4.csv  
  inflating: data/rejected_2007_to_2018Q4.csv.gz  
  inflating: data/rejected_2007_to_2018q4.csv/rejected_2007_to_2018Q4.csv  


In [25]:
import os
os.listdir("data/accepted_2007_to_2018q4.csv")


['accepted_2007_to_2018Q4.csv']

In [26]:
import os
os.listdir("data")


['accepted_2007_to_2018q4.csv',
 'rejected_2007_to_2018Q4.csv.gz',
 'accepted_2007_to_2018Q4.csv.gz',
 'rejected_2007_to_2018q4.csv']

In [27]:
for root, dirs, files in os.walk("data"):
    for file in files:
        if file.endswith(".csv"):
            print(os.path.join(root, file))


data/accepted_2007_to_2018q4.csv/accepted_2007_to_2018Q4.csv
data/rejected_2007_to_2018q4.csv/rejected_2007_to_2018Q4.csv


In [28]:
import pandas as pd

df = pd.read_csv(
    "data/accepted_2007_to_2018q4.csv/accepted_2007_to_2018Q4.csv",
    low_memory=False
)

df.head()


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
important_cols = [
    'loan_amnt', 'term', 'int_rate', 'installment',
    'grade', 'sub_grade', 'emp_length',
    'home_ownership', 'annual_inc',
    'purpose', 'dti', 'loan_status'
]

df = df[important_cols]
df.head()


,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,purpose,dti,loan_status
0,3600.0,36 months,13.99,123.03,C,C4,10+ years,MORTGAGE,55000.0,debt_consolidation,5.91,Fully Paid
1,24700.0,36 months,11.99,820.28,C,C1,10+ years,MORTGAGE,65000.0,small_business,16.06,Fully Paid
2,20000.0,60 months,10.78,432.66,B,B4,10+ years,MORTGAGE,63000.0,home_improvement,10.78,Fully Paid
3,35000.0,60 months,14.85,829.90,C,C5,10+ years,MORTGAGE,110000.0,debt_consolidation,17.06,Current
4,10400.0,60 months,22.45,289.91,F,F1,3 years,MORTGAGE,104433.0,major_purchase,25.37,Fully Paid


In [30]:
df = df[df['loan_status'].isin(['Fully Paid', 'Charged Off'])]

df['loan_status'] = df['loan_status'].map({
    'Fully Paid': 0,
    'Charged Off': 1
})

df['loan_status'].value_counts()


,count
loan_status,
0,1076751
1,268559


In [34]:
# Convert int_rate safely
if df['int_rate'].dtype == 'object':
    df['int_rate'] = df['int_rate'].str.replace('%', '').astype(float)
else:
    df['int_rate'] = df['int_rate'].astype(float)
# Fill missing numeric values
df.fillna(df.median(numeric_only=True), inplace=True)


In [35]:
import pandas as pd

df = pd.get_dummies(df, drop_first=True)
df.shape


(1345310, 75)

In [37]:
from sklearn.model_selection import train_test_split

X = df.drop('loan_status', axis=1)
y = df['loan_status']

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [41]:
from sklearn.linear_model import SGDClassifier

sgd = SGDClassifier(
    loss='log_loss',
    max_iter=1000,
    class_weight='balanced',
    random_state=42
)

sgd.fit(X_train, y_train)


SGDClassifier(class_weight='balanced', loss='log_loss', random_state=42)

In [43]:
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix

# Predictions
y_pred = sgd.predict(X_test)

# SGD does NOT always have predict_proba
# Use decision_function instead
y_scores = sgd.decision_function(X_test)

print(classification_report(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_scores))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.80      0.99      0.89    215350
           1       0.35      0.01      0.03     53712

    accuracy                           0.80    269062
   macro avg       0.58      0.50      0.46    269062
weighted avg       0.71      0.80      0.72    269062

ROC-AUC: 0.5897947178353864
Confusion Matrix:
 [[214021   1329]
 [ 52994    718]]


In [44]:
!pip install -q xgboost


In [46]:
# Clean column names for XGBoost
X_train.columns = (
    X_train.columns
    .str.replace('[', '', regex=False)
    .str.replace(']', '', regex=False)
    .str.replace('<', 'lt', regex=False)
    .str.replace('>', 'gt', regex=False)
    .str.replace(' ', '_', regex=False)
)

X_test.columns = X_train.columns  # keep same order


In [47]:
from xgboost import XGBClassifier

xgb = XGBClassifier(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric='auc',
    random_state=42
)

xgb.fit(X_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='auc', feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, ...)

In [48]:
from sklearn.metrics import classification_report, roc_auc_score

y_pred_xgb = xgb.predict(X_test)
y_prob_xgb = xgb.predict_proba(X_test)[:, 1]

print(classification_report(y_test, y_pred_xgb))
print("XGBoost ROC-AUC:", roc_auc_score(y_test, y_prob_xgb))


              precision    recall  f1-score   support

           0       0.81      0.99      0.89    215350
           1       0.55      0.06      0.10     53712

    accuracy                           0.80    269062
   macro avg       0.68      0.52      0.49    269062
weighted avg       0.76      0.80      0.73    269062

XGBoost ROC-AUC: 0.7119722222481583


In [49]:
def risk_category(prob):
    if prob >= 0.7:
        return "High Risk"
    elif prob >= 0.4:
        return "Medium Risk"
    else:
        return "Low Risk"

sample_prob = y_prob_xgb[0]
risk_category(sample_prob)


'Low Risk'

In [50]:
import joblib

joblib.dump(xgb, "credit_risk_model.pkl")
joblib.dump(X.columns, "model_features.pkl")


['model_features.pkl']

In [51]:
# Load and predict like real-time system
model = joblib.load("credit_risk_model.pkl")
features = joblib.load("model_features.pkl")

sample_loan = X_test.iloc[0:1]
prob = model.predict_proba(sample_loan)[0][1]

print("Default Probability:", round(prob, 3))
print("Risk Level:", risk_category(prob))


Default Probability: 0.089
Risk Level: Low Risk


In [52]:
for i in range(5):
    sample = X_test.iloc[[i]]
    prob = model.predict_proba(sample)[0][1]
    print(i, round(prob,3), risk_category(prob))


0 0.089 Low Risk
1 0.127 Low Risk
2 0.097 Low Risk
3 0.2 Low Risk
4 0.21 Low Risk


In [53]:
def bank_decision(prob):
    if prob >= 0.7:
        return "REJECT"
    elif prob >= 0.4:
        return "MANUAL REVIEW"
    else:
        return "APPROVE"


In [54]:
bank_decision(prob)


'APPROVE'

In [55]:
import pandas as pd

feature_importance = pd.Series(
    model.feature_importances_,
    index=X.columns
).sort_values(ascending=False)

feature_importance.head(10)


,0
int_rate,0.163501
term_ 60 months,0.130370
grade_D,0.092795
grade_C,0.090436
grade_B,0.060650
home_ownership_MORTGAGE,0.037641
grade_E,0.033566
home_ownership_RENT,0.033474
sub_grade_C2,0.024726
sub_grade_C4,0.019154


SyntaxError: invalid character '│' (U+2502) (ipython-input-3887539557.py, line 2)